In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')
from visualize import visualize_svm as Visualize

#
sys.path.append('../..')
from svm import svm as SVM


Autosaving every 180 seconds


<ipython-input-1-a90fe7f1d8b3>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#################################################
############# HELPER FUNCTIONS ##################
#################################################
# 
import scipy
from statsmodels.stats.multitest import multipletests

#
def load_supersessions(animal_id, lockout):
    fname = os.path.join('/media/cat/4TBSSD/yuki/',
                         animal_id,
                         'super_sessions',
                         'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    trials = np.vstack(data['trials'])
    random = np.vstack(data['random'])
    
    return trials, random

# 
def filter_trace(trace, smooth_window=30):

    box = np.ones(smooth_window)/smooth_window
    trace_smooth = np.convolve(trace, box, mode='valid')

    return trace_smooth


def compute_significance(data):

    #
    significance = 0.05

    #
    sig = []
    for k in range(data.shape[0]):

        #
        res = scipy.stats.ttest_1samp(data[k], 0.5)

        sig.append(res[1])


    sig_save = np.array(sig).copy()

    # multiple hypothesis test Benjamini-Hockberg
    temp = np.array(sig)
    temp2 = multipletests(temp, 
                          alpha=significance, 
                          method='fdr_bh')
    sig = temp2[1]

    # find earliest significant;
    edt = 0
    for k in range(sig.shape[0]-1,0,-1):
        if sig[k]<=significance:
            edt = k+1
        else:
            break

    # 
    edtsec = -(sig.shape[0]-edt)/30.

    # 
    return edtsec

In [5]:
################################################################
#### SUBSAMPLE 3SEC LOCKOUT TRIALS TO MATCH 30SEC LOCKOUTS #####
################################################################
# 
animal_ids = ['IA1','IA2','IA3',
              'IJ1','IJ2','AQ2']
lockout1 = 12

#
for animal_id in animal_ids:
    # animal_id = 'AQ2'

    print ("animal id: ", animal_id)
    # load 30sec lockout 
    trials30, random30 = load_supersessions(animal_id, lockout1)
    print ("trials30: ", trials30.shape)

    # load 3sec lockouts
    lockout2 = 0  # the 3sec lockout data is saved as 0seclockout
    trials3, random3 = load_supersessions(animal_id, lockout2)
    print ("trials3: ", trials3.shape)

    # subsample 3sec lockout data
    idx = np.int32(np.linspace(0,trials3.shape[0]-1,trials30.shape[0]))
    trials3_sub = trials3[idx].transpose(0,2,1)
    random3_sub = random3[idx].transpose(0,2,1)
    print ("trials3_sub: ", trials3_sub.shape)

    # 
    svm = SVM.PredictSVMChoiceSuperSession()
    svm.xvalidation = 10
    svm.parallel = True

    # window parameters
    svm.window = 30              # seconds of prediction window, backwards in time in seconds; e.g. -30sec ... +30sec
    svm.lockout_window = 10      # time for locking out other pulls 
    svm.sliding_window = 30      # number of frames in sliding window

    # pca params
    svm.pca_flag = True
    svm.pca_var = 0.95        # this is default for now for nComp = 20

    # svm parameters
    svm.xvalidation = 10      # KFold xvalidation step
    svm.data_split = 0.8      # split 80/20 data for training
    svm.method = 'sigmoid'    # method used for computing SVM

    # run-time parameters
    svm.parallel = True
    svm.n_cores = svm.xvalidation
    svm.min_trials = 30
    svm.overwrite = True
    svm.gpu_flag = False

    #
    fname_out = os.path.join('/media/cat/4TBSSD/yuki/',
                             animal_id,
                             'super_sessions',
                             'alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats_super_res_subsample_'+
                             str(lockout1)+'matched_lockout.npz')

    #
    svm.predict3(trials3_sub, random3_sub, fname_out)



animal id:  IA1
trials30:  (601, 1800, 16)
trials3:  (1351, 1800, 16)
trials3_sub:  (601, 16, 1800)
   start:  0  end:  200  /  601
   start:  50  end:  250  /  601
   start:  100  end:  300  /  601
   start:  150  end:  350  /  601
   start:  200  end:  400  /  601
   start:  250  end:  450  /  601
   start:  300  end:  500  /  601
   start:  350  end:  550  /  601
   start:  400  end:  600  /  601
   start:  450  end:  650  /  601
   start:  500  end:  700  /  601
   start:  550  end:  750  /  601
animal id:  IA2
trials30:  (728, 1800, 16)
trials3:  (1704, 1800, 16)
trials3_sub:  (728, 16, 1800)
   start:  0  end:  200  /  728
   start:  50  end:  250  /  728
   start:  100  end:  300  /  728
   start:  150  end:  350  /  728
   start:  200  end:  400  /  728
   start:  250  end:  450  /  728
   start:  300  end:  500  /  728
   start:  350  end:  550  /  728
   start:  400  end:  600  /  728
   start:  450  end:  650  /  728
   start:  500  end:  700  /  728
   start:  550  end:  75